In [18]:
import numpy as np
import time

In [19]:
# Indices:
#        ┌──┬──┐
#        │ 0│ 1│
#        ├──┼──┤
#        │ 2│ 3│
#  ┌──┬──┼──┼──┼──┬──┬──┬──┐
#  │16│17│ 8│ 9│ 4│ 5│20│21│
#  ├──┼──┼──┼──┼──┼──┼──┼──┤
#  │18│19│10│11│ 6│ 7│22│23│
#  └──┴──┼──┼──┼──┴──┴──┴──┘
#        │12│13│
#        ├──┼──┤
#        │14│15│
#        └──┴──┘

def state2nice(s):
    return f'''
        ┌──┬──┐
        │{ s[0]:2}│{ s[1]:2}│
        ├──┼──┤
        │{ s[2]:2}│{ s[3]:2}│
  ┌──┬──┼──┼──┼──┬──┬──┬──┐
  │{s[16]:2}│{s[17]:2}│{ s[8]:2}│{ s[9]:2}│{ s[4]:2}│{ s[5]:2}│{s[20]:2}│{s[21]:2}│
  ├──┼──┼──┼──┼──┼──┼──┼──┤
  │{s[18]:2}│{s[19]:2}│{s[10]:2}│{s[11]:2}│{ s[6]:2}│{ s[7]:2}│{s[22]:2}│{s[23]:2}│
  └──┴──┼──┼──┼──┴──┴──┴──┘
        │{s[12]:2}│{s[13]:2}│
        ├──┼──┤
        │{s[14]:2}│{s[15]:2}│
        └──┴──┘
        '''

perms = {
      "I":  [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
      "CR": [ 8,  9, 10, 11,  6,  4,  7,  5, 12, 13, 14, 15, 23, 22, 21, 20, 17, 19, 16, 18,  3,  2,  1,  0],
      "CU": [ 2,  0,  3,  1, 20, 21, 22, 23,  4,  5,  6,  7, 13, 15, 12, 14,  8,  9, 10, 11, 16, 17, 18, 19],
      "R":  [ 0,  9,  2, 11,  6,  4,  7,  5,  8, 13, 10, 15, 12, 22, 14, 20, 16, 17, 18, 19,  3, 21,  1, 23],
}

pns = [f"{d}{t}" for d in ['R', 'U', 'F'] for t in ['','2',"'"]]
# pns = [f"{d}{t}" for d in ['R', 'U', 'F'] for t in ['']]

def apply(pns):
    t = None
    for n in pns:
        p = perms[n]
        if t is None:
            t = p
            continue
        t = t[p]
    return t
    
dtype = np.ushort
dtype = None
perms = {k: np.array(v, dtype=dtype) for k, v in perms.items()}

In [20]:
# AUGMENT ROTATIONS
# full cube rotations
perms["CR2"] = apply(["CR", "CR"])
perms["CR'"] = apply(["CR2", "CR"])

perms["CU2"] = apply(["CU", "CU"])
perms["CU'"] = apply(["CU2", "CU"])

perms["CF'"] = apply(["CU", "CR", "CU'"])
perms["CF2"] = apply(["CF'", "CF'"])
perms["CF"] = apply(["CF2", "CF'"])

# half cube rotations
perms["R2"] = apply(["R", "R"])
perms["R'"] = apply(["R2", "R"])

perms["F"] = apply(["CU'", "R", "CU"])
perms["F2"] = apply(["F", "F"])
perms["F'"] = apply(["F2", "F"])

perms["U"] = apply(["CF", "R", "CF'"])
perms["U2"] = apply(["U", "U"])
perms["U'"] = apply(["U2", "U"])

# print(state2nice(perms["U"]))

In [21]:
class Cube(object):
    state = None
    
    def __init__(self, state = None):
        if state is None:
            state = np.array(list(range(24))).flatten()
        self.state = state

    def __str__(self):
        return state2nice(self.state)

    def apply(self, name):
        if name not in perms:
            raise Exception(f'`{name}`no valid permutation name provided!')
        self.state = self.state[perms[name]]
        return self

    def rotate(self, d=None, r=1):
        for i in range(r%4):
            self.apply(f'C{d}')
        return self

    def turn(self, d=None, r=1):
        for i in range(r%4):
            self.apply(d)
        return self

In [22]:
a = Cube()
print(a)
for d in ['R', 'U'] * 15:
    a.turn(d)
print(a)


        ┌──┬──┐
        │ 0│ 1│
        ├──┼──┤
        │ 2│ 3│
  ┌──┬──┼──┼──┼──┬──┬──┬──┐
  │16│17│ 8│ 9│ 4│ 5│20│21│
  ├──┼──┼──┼──┼──┼──┼──┼──┤
  │18│19│10│11│ 6│ 7│22│23│
  └──┴──┼──┼──┼──┴──┴──┴──┘
        │12│13│
        ├──┼──┤
        │14│15│
        └──┴──┘
        

        ┌──┬──┐
        │ 0│ 1│
        ├──┼──┤
        │ 2│ 3│
  ┌──┬──┼──┼──┼──┬──┬──┬──┐
  │16│17│ 8│ 9│ 4│ 5│20│21│
  ├──┼──┼──┼──┼──┼──┼──┼──┤
  │18│19│10│11│ 6│ 7│22│23│
  └──┴──┼──┼──┼──┴──┴──┴──┘
        │12│13│
        ├──┼──┤
        │14│15│
        └──┴──┘
        


In [24]:
def s2k(n):
    return n.tobytes()
    # return str(n.tolist())

def explore_step(sts, i):
    todo = []
    for s in sts:
        for pn in pns:
            sn = s[perms[pn]]
            k = s2k(sn)
            if k in states:
                continue
            states[k] = i + 1
            todo.append(sn)
    return todo

states = {
    s2k(perms['I']): 0
}
to_explore = [perms['I']]

start = time.time()
for it in range(0, 100):
    print(f"- It. {it:2}: {len(to_explore):7}")
    to_explore = explore_step(to_explore, it)
    if len(to_explore) == 0:
        print(f'Finished, Count: {len(states)}')
        break
end = time.time()
print(f"{round(end - start)} seconds elapsed.")

- Iteration  0:       1
- Iteration  1:       9
- Iteration  2:      54
- Iteration  3:     321
- Iteration  4:    1847
- Iteration  5:    9992
- Iteration  6:   50136
- Iteration  7:  227536
- Iteration  8:  870072
- Iteration  9: 1887748
- Iteration 10:  623800
- Iteration 11:    2644
Finished, Count: 3674160
30 seconds elapsed.
